### 函数式API简介

In [1]:
from keras import Input, layers

input_tensor = Input(shape=(32,))

dense = layers.Dense(32, activation='relu')

output_tensor = dense(input_tensor)

Using TensorFlow backend.


In [2]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

# Sequential模型
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

# 对应函数式API实现
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

# Model类将输入张量和输出张量转换为一个模型
model = Model(input_tensor, output_tensor)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [3]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# 生成用于训练的虚构 Numpy 数据
import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=10, batch_size=128)
score = model.evaluate(x_train, y_train)
score

Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 11.6107
Epoch 2/10
1000/1000 [==============================] - 0s 33us/step - loss: 11.5616
Epoch 3/10
1000/1000 [==============================] - 0s 32us/step - loss: 11.5535
Epoch 4/10
1000/1000 [==============================] - 0s 32us/step - loss: 11.5496
Epoch 5/10
1000/1000 [==============================] - 0s 32us/step - loss: 11.5469
Epoch 6/10
1000/1000 [==============================] - 0s 32us/step - loss: 11.5444
Epoch 7/10
1000/1000 [==============================] - 0s 31us/step - loss: 11.5426
Epoch 8/10
1000/1000 [==============================] - 0s 33us/step - loss: 11.5407
Epoch 9/10
1000/1000 [==============================] - 0s 33us/step - loss: 11.5392
Epoch 10/10
1000/1000 [==============================] - 0s 66us/step


11.534444198608398

### 多输入模型

#### 用函数式API实现双输入问答模型

In [4]:
from keras.models import Model
from keras import layers
from keras import Input
import keras

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 5000

# 文本输入是一个长度可变的整数序列。注意，你可以选择对输入进行命名
text_input = Input(shape=(None,), dtype='int32', name='text')

# 将输入嵌入长度为64的向量
embedded_text = layers.Embedding(
    text_vocabulary_size, 64)(text_input)

# 利用LSTM将向量编码为单个向量
encoded_text = layers.LSTM(32)(embedded_text)

# 对问题进行相同的处理
question_input = Input(shape=(None,), 
                       dtype='int32', 
                       name='question')

embedded_question = layers.Embedding(
    question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# 将编码后的问题和文本链接起来
concatenated = layers.concatenate([encoded_text, encoded_question],
                                  axis=-1)

# 在上面添加一个 softmax 分类器
answer = layers.Dense(answer_vocabulary_size,
                      activation='softmax')(concatenated)

# 在模型实例化时，指定两个输入和输出
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text (InputLayer)               (None, None)         0                                            
__________________________________________________________________________________________________
question (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 64)     640000      text[0][0]                       
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 32)     320000      question[0][0]                   
__________________________________________________________________________________________________
lstm_1 (LS

#### 将数据输入到多输入模型

In [5]:
import numpy as np

num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size,
                         size=(num_samples, max_length))

question = np.random.randint(1, question_vocabulary_size,
                             size=(num_samples, max_length))
answers = np.random.randint(answer_vocabulary_size, size=(num_samples))
answers = keras.utils.to_categorical(answers, answer_vocabulary_size)

# 使用输入组成的列表来拟合
model.fit([text, question], answers, epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 4s 4ms/step - loss: 8.5173 - acc: 0.0000e+00
Epoch 2/10
1000/1000 [==============================] - 2s 2ms/step - loss: 8.4898 - acc: 0.0090
Epoch 3/10
1000/1000 [==============================] - 2s 2ms/step - loss: 8.1850 - acc: 0.0030
Epoch 4/10
1000/1000 [==============================] - 2s 2ms/step - loss: 7.7293 - acc: 0.0040
Epoch 5/10
1000/1000 [==============================] - 2s 2ms/step - loss: 7.4272 - acc: 0.0040
Epoch 6/10
1000/1000 [==============================] - 3s 3ms/step - loss: 7.2428 - acc: 0.0040
Epoch 7/10
1000/1000 [==============================] - 2s 2ms/step - loss: 7.1219 - acc: 0.0040
Epoch 8/10
1000/1000 [==============================] - 2s 2ms/step - loss: 7.0420 - acc: 0.0040
Epoch 9/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.9901 - acc: 0.0040
Epoch 10/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.9540 - acc: 0.0040


In [6]:
# 使用输入组成的字典来拟合（只有对输入进行命名之后才能使用这种方法）
model.fit({'text': text, 'question': question}, answers,
          epochs=10, batch_size=128)

Epoch 1/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.9265 - acc: 0.0040
Epoch 2/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.9065 - acc: 0.0040
Epoch 3/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.8900 - acc: 0.0040
Epoch 4/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.8775 - acc: 0.0040
Epoch 5/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.8678 - acc: 0.0040
Epoch 6/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.8601 - acc: 0.0040
Epoch 7/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.8536 - acc: 0.0040
Epoch 8/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.8475 - acc: 0.0040
Epoch 9/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.8418 - acc: 0.0040
Epoch 10/10
1000/1000 [==============================] - 2s 2ms/step - loss: 6.8338 - acc: 0.0040


### 多输出模型

#### 用函数式API实现一个三输出模型

In [14]:
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

# 注意，输出层都具有名称
age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups,
                                 activation='softmax',
                                 name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input, 
              [age_prediction, income_prediction, gender_prediction])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50000)  3200000     posts[0][0]                      
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 32)     8000032     embedding_1[0][0]                
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, None, 32)     0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
conv1d_2 (

#### 多输出模型的编译选项：多重损失

In [15]:
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

# 与上述方法等效，只有输出层具有名称时才能采用这种写法）
model.compile(optimizer='rmsprop',
              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'})

#### 多输出模型的编译选项：损失加权

In [16]:
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'], 
              loss_weights=[0.25, 1., 10.])

# 与上述方法等效，只有输出层具有名称时才能采用这种写法）
model.compile(optimizer='rmsprop',
              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'}, 
              loss_weights={'age': 0.25,
                            'income': 1.,
                            'gender': 10.})

#### 将数据输入到多输出模型中

> No data, don't run.

In [ ]:
model.fit(posts, [age_targets, income_targets, gender_targets],
          epochs=10, batch_size=64)

# 与上述写法等效（只有输出层具有名称时才能采用这种写法）
model.fit(posts, {'age': age_targets,
                  'income': income_targets,
                  'gender': gender_targets},
          epochs=10, batch_size=32)

### 层组成的有向无环图

#### Inception 模块

In [25]:
from keras import layers
from keras import Input

x = Input(shape=(None, None, 3))

# 每个分支都有相同的步幅值（2），这对于保持所有分支输出具有
# 相同的尺寸是很有必要的，这样你才能将它们连接在一起
branch_a = layers.Conv2D(128, 1,
                         activation='relu', strides=2)(x)

# 在这个分支中，空间卷积层用到和步幅
branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_b)

# 在这个分支中，平均池化层用到了步幅
branch_c = layers.AveragePooling2D(3, strides=2)(x)
branch_c = layers.Conv2D(128, 3, activation='relu')(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_d)

output = layers.concatenate(
    [branch_a, branch_b, branch_c, branch_d], axis=-1)

#### 恒等残差连接

In [28]:
from keras import layers

x = Input(shape=(None, None, 3))
# 对 x 进行变换
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)

# 将原始 x 与输出特征相加
y = layers.add([y, x])

#### 线性残差连接

In [31]:
from keras import layers

x = Input(shape=(None, None, 3))
# 对 x 进行变换
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(3, strides=2)(y)

# 使用 1x1 卷积，将原始 x 张量线性下采样为与 y 具有相同的形状
residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)

# 将残差张量与输出特征相加
y = layers.add([y, residual])

### 共享层权重

In [33]:
from keras import layers
from keras import Input
from keras.models import Model

# 将一个LSTM层实例化
lstm = layers.LSTM(32)

# 构建模型的左分支：输入是长度128的向量组成的变长序列
left_input = Input(shape=(None, 128))
left_output = lstm(left_input)

# 构建模型的右分支：如果调用已有的层实例，那么会重复使用它的权重
right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

# 在上面构建一个分类器
merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)

model = Model([left_input, right_input], predictions)

### 将模型作为层

In [34]:
from keras import layers
from keras import applications
from keras import Input

# 图像处理基础模型是 Xception 网络（只包含卷积基）
xception_base = applications.Xception(weights=None,
                                      include_top=False)

# 输入数 250x250 的 RGB 图像
left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

# 对相同的视觉模型调用两次
left_features = xception_base(left_input)
right_features = xception_base(right_input)

# 合并后的特征包含来自左右两个视觉输入中的信息
merged_features = layers.concatenate(
    [left_features, right_features], axis=-1)